## 1、快递费

#### 1.0 最终和供应商确认版账单:文件|路径

<font color = "red"/> 待定</font>
```


class YaAo():
    def __init__(self,input_path,input_file):
        self.input_path = input_path 
        self.input_file = input_file 
    
    def first_read(self,Tier,Vendor):
        self.df = pd.read_excel(os.path.join(input_path,input_file),sheet_name="2.1 快递费",header=1)
        self.df["重量段"] = df["总重量 (kg)"].map(Tier)
        self.df["物流公司"] = df["物流公司"].map(Vendor)
        self.df_pivot = df.pivot_table(index=["月份","店铺/货主","仓库区域","省份","重量段","物流公司"],
                          values=["物流单号/运单号","原始单号/外部订单号","计费重量","总重量 (kg)","快递费(总)","货品数量/拣货数量"],
                          aggfunc={"物流单号/运单号":"count","原始单号/外部订单号":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","快递费(总)":"sum",
                                  '货品数量/拣货数量':"sum"})
        self.df_pivot.rename(columns={"原始单号/外部订单号":"订单量","物流单号/运单号":"包裹量"},inplace=True)
        self.df_pivot = df_pivot.reset_index()
        
        return df_pivot
    def second_to_excel(self,output):
        self.to_excel(os.path.join(output,input_file))
        
```

In [31]:
#【雅澳_上海_B2C】物流费用账单 202203 0414  v2.0 KZC反馈--雅澳5.20确认.xlsx    （原名称）


input_path = r"D:\我的坚果云\Work  Share\许程晨\4、In Process\2、外仓供应商账单\Final"
input_file = r"【雅澳_上海_B2C】物流费用账单 202201 v1.4.xlsx"

#### 1.1 读入文件

In [32]:
import os
import pandas as pd
import math

In [33]:
df = pd.read_excel(os.path.join(input_path,input_file),sheet_name="2.2 订单处理费、快递费、包材费",header=1)
df[:5]

C:\Users\ChengChen Xu\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,运输类型,仓库区域,仓库名称,店铺/货主,订单类型,订单日期,发货日期,订单编号/包裹号,出库单号/销售订单编码,物流单号/运单号,...,续重 (元/kg),快递费,耗材编码01,快递耗材01,耗材单价,耗材数量,耗材编码02,快递耗材02,耗材单价.1,耗材数量.1
0,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,福佳旗舰店,销售订单,2022-01-26,2022-01-26,JY202201269923,CK2022012611640,432368220610500,...,0.0,2.6,NaN,NaN,0,1,NaN,NaN,0,NaN
1,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201262748,CK2022012610298,432368102801135,...,0.0,2.6,NaN,NaN,0,1,NaN,NaN,0,NaN
2,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201256686,CK2022012512305-1,432368102801046,...,0.0,2.6,NaN,NaN,0,1,NaN,NaN,0,NaN
3,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201262739,CK2022012610297,432368102800950,...,0.0,2.6,NaN,NaN,0,1,NaN,NaN,0,NaN
4,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201256677,CK2022012512296-1,432368102800862,...,0.0,3.6,NaN,NaN,0,1,NaN,NaN,0,NaN


In [34]:
df.columns

Index(['运输类型', '仓库区域', '仓库名称', '店铺/货主', '订单类型', '订单日期', '发货日期', '订单编号/包裹号',
       '出库单号/销售订单编码', '物流单号/运单号', '原始单号/外部订单号', '商品名称', '商品', '货品数量',
       '总体积 (m³)', '总重量 (kg)', '客服备注', '物流公司', '省份', '城市', '区县', '包材编码',
       '包装耗材', '包装数量', '包装单价', '账单类型', '包材费', '重量段 (订单处理)', '首件 (元)', '续件 (元)',
       '订单处理费', '重量段 (快递费)', '首重 (kg)', '续重 (kg)', '首重 (元)', '续重 (元/kg)',
       '快递费', '耗材编码01', '快递耗材01', '耗材单价', '耗材数量', '耗材编码02', '快递耗材02', '耗材单价.1',
       '耗材数量.1'],
      dtype='object')

#### 1.2 定义重量段

In [35]:
def Tier(weight):
    if weight > 10 :
        Tier = "X >10kg"
    elif weight > 5 :
        Tier = "X >5kg,≤10kg"
    elif weight > 3 :
        Tier = "X >3kg,≤5kg"
    elif weight > 2 :
        Tier = "X >2kg, ≤3kg"
    elif weight > 1 :
        Tier = "X >1kg, ≤2kg"
    elif weight > 0.5 :
        Tier = "X >0.5kg, ≤1kg"
    elif weight > 0.3 :
        Tier = "X >0.3kg, ≤0.5kg"
    else :
        Tier = "X ≤0.3kg"
    return Tier 

In [36]:
df["重量段"] = df["总重量 (kg)"].map(Tier)
df["计费重量"] = df["总重量 (kg)"].map(math.ceil)
df["订单量"] = 1
df["订单量"][df["原始单号/外部订单号"].duplicated()==True] = 0

C:\Users\CHENGC~1\AppData\Local\Temp/ipykernel_14288/951243271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["订单量"][df["原始单号/外部订单号"].duplicated()==True] = 0


In [37]:
df["原始单号/外部订单号"].duplicated().value_counts()

False    30098
True       144
Name: 原始单号/外部订单号, dtype: int64

#### 1.3 定义物流公司

<font color="red"> 待定 </font>
```
def Vendor(x):
    if x == "中通标快（普冷）":
        Vendor = "中通快递"
    elif x == "顺丰标快（普冷）":
        Vendor = "顺丰快递"
    return Vendor 

df["物流公司"] = df["物流公司"].map(Vendor)
```

#### 1.4 Pivot

In [38]:
df_pivot = df.pivot_table(index=["店铺/货主","仓库区域","省份","重量段","物流公司"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","快递费","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","快递费":"sum",
                                  '货品数量':"sum"},margins=True)

df_pivot

快递费   总重量 (kg)  物流单号/运单号  \
店铺/货主    仓库区域 省份      重量段              物流公司                                 
Mikes旗舰店 上海   上海      X ≤0.3kg         上海雅澳      7.2      0.421         3   
              安徽省     X >2kg, ≤3kg     上海雅澳      3.6      2.403         1   
              山东省     X ≤0.3kg         上海雅澳      2.4      0.075         1   
              广东省     X ≤0.3kg         上海雅澳      2.4      0.251         1   
              广西壮族自治区 X ≤0.3kg         上海雅澳      2.4      0.074         1   
...                                              ...        ...       ...   
鹅岛啤酒旗舰店  上海   湖北省     X >0.3kg, ≤0.5kg 上海雅澳      2.5      0.415         1   
              福建省     X >0.3kg, ≤0.5kg 上海雅澳     12.5      2.075         5   
                      X ≤0.3kg         上海雅澳      2.4      0.217         1   
              辽宁省     X >0.3kg, ≤0.5kg 上海雅澳      2.5      0.415         1   
All                                          79010.2  16672.947     30242   

                                              计费重量    订单量   货品数量  
店铺/货主    仓库区域 省份      重量段              物流公司                       
Mikes旗舰店 上海   上海      X ≤0.3kg         上海雅澳      3      3      6  
              安徽省     X >2kg, ≤3kg     上海雅澳      3      1      6  
              山东省     X ≤0.3kg         上海雅澳      1      1      1  
              广东省     X ≤0.3kg         上海雅澳      1      1      2  
              广西壮族自治区 X ≤0.3kg         上海雅澳      1      1      1  
...                                            ...    ...    ...  
鹅岛啤酒旗舰店  上海   湖北省     X >0.3kg, ≤0.5kg 上海雅澳      1      1      1  
              福建省     X >0.3kg, ≤0.5kg 上海雅澳      5      5      5  
                      X ≤0.3kg         上海雅澳      1      1      1  
              辽宁省     X >0.3kg, ≤0.5kg 上海雅澳      1      1      1  
All                                          34952  30098  48113  

[870 rows x 6 columns]

In [39]:
df_pivot.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)

In [40]:
df_pivot = df_pivot.reset_index()

df_pivot

,店铺/货主,仓库区域,省份,重量段,物流公司,快递费,总重量 (kg),包裹量,计费重量,订单量,货品数量
0,Mikes旗舰店,上海,上海,X ≤0.3kg,上海雅澳,7.2,0.421,3,3,3,6
1,Mikes旗舰店,上海,安徽省,"X >2kg, ≤3kg",上海雅澳,3.6,2.403,1,3,1,6
2,Mikes旗舰店,上海,山东省,X ≤0.3kg,上海雅澳,2.4,0.075,1,1,1,1
3,Mikes旗舰店,上海,广东省,X ≤0.3kg,上海雅澳,2.4,0.251,1,1,1,2
4,Mikes旗舰店,上海,广西壮族自治区,X ≤0.3kg,上海雅澳,2.4,0.074,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
865,鹅岛啤酒旗舰店,上海,湖北省,"X >0.3kg, ≤0.5kg",上海雅澳,2.5,0.415,1,1,1,1
866,鹅岛啤酒旗舰店,上海,福建省,"X >0.3kg, ≤0.5kg",上海雅澳,12.5,2.075,5,5,5,5
867,鹅岛啤酒旗舰店,上海,福建省,X ≤0.3kg,上海雅澳,2.4,0.217,1,1,1,1
868,鹅岛啤酒旗舰店,上海,辽宁省,"X >0.3kg, ≤0.5kg",上海雅澳,2.5,0.415,1,1,1,1


#### 1.5 输出pivot

In [41]:
output = r"D:\我的坚果云\Work  Share\许程晨\5、Billing Summary\8、VLC_2022"

In [42]:
df_pivot.to_excel(os.path.join(output,"快递费pivot"+input_file))

## 2、订单处理费

In [43]:
df

,运输类型,仓库区域,仓库名称,店铺/货主,订单类型,订单日期,发货日期,订单编号/包裹号,出库单号/销售订单编码,物流单号/运单号,...,快递耗材01,耗材单价,耗材数量,耗材编码02,快递耗材02,耗材单价.1,耗材数量.1,重量段,计费重量,订单量
0,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,福佳旗舰店,销售订单,2022-01-26,2022-01-26,JY202201269923,CK2022012611640,432368220610500,...,NaN,0,1,NaN,NaN,0,NaN,"X >0.5kg, ≤1kg",1,1
1,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201262748,CK2022012610298,432368102801135,...,NaN,0,1,NaN,NaN,0,NaN,"X >0.5kg, ≤1kg",1,1
2,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201256686,CK2022012512305-1,432368102801046,...,NaN,0,1,NaN,NaN,0,NaN,"X >0.5kg, ≤1kg",1,1
3,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201262739,CK2022012610297,432368102800950,...,NaN,0,1,NaN,NaN,0,NaN,"X >0.5kg, ≤1kg",1,1
4,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,三得利预调酒官方旗舰店,销售订单,2022-01-26,2022-01-26,JY202201256677,CK2022012512296-1,432368102800862,...,NaN,0,1,NaN,NaN,0,NaN,"X >0.5kg, ≤1kg",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30237,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,奈森克林旗舰店,销售订单,2021-12-31,2021-12-31,JY2021123034911,CK202112314335,432306229870110,...,NaN,0,1,NaN,NaN,0,NaN,X ≤0.3kg,1,1
30238,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,奈森克林旗舰店,销售订单,2021-12-31,2021-12-31,JY2021123031005,CK202112314333,432306229870022,...,NaN,0,1,NaN,NaN,0,NaN,X ≤0.3kg,1,1
30239,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,奈森克林旗舰店,销售订单,2021-12-30,2021-12-30,JY202112306882,CK202112306189-2,432303882870464,...,NaN,0,1,NaN,NaN,0,NaN,X ≤0.3kg,1,1
30240,B2C-正向 (常规),上海,雅澳_上海_B2C正品仓,奈森克林旗舰店,销售订单,2021-12-30,2021-12-30,JY202112307201,CK202112306430-2,432303882870376,...,NaN,0,1,NaN,NaN,0,NaN,X ≤0.3kg,1,1


In [44]:
df_pivot_order_handing = df.pivot_table(index=["店铺/货主","仓库区域","重量段"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","订单处理费","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","订单处理费":"sum",
                                  '货品数量':"sum"},margins=True)

df_pivot_order_handing
df_pivot_order_handing.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)
df_pivot_order_handing = df_pivot_order_handing.reset_index()

df_pivot_order_handing

,店铺/货主,仓库区域,重量段,总重量 (kg),包裹量,计费重量,订单处理费,订单量,货品数量
0,Mikes旗舰店,上海,"X >0.3kg, ≤0.5kg",1.358,3,3,2.1,3,8
1,Mikes旗舰店,上海,"X >2kg, ≤3kg",2.403,1,3,1.0,1,6
2,Mikes旗舰店,上海,X ≤0.3kg,1.223,9,9,6.3,9,13
3,franziskaner旗舰店,上海,"X >0.3kg, ≤0.5kg",3.110,10,10,7.0,10,10
4,franziskaner旗舰店,上海,"X >0.5kg, ≤1kg",636.481,1033,1033,723.1,1033,1033
...,...,...,...,...,...,...,...,...,...
71,雪津旗舰店,上海,X ≤0.3kg,4.182,15,15,10.5,15,15
72,鹅岛啤酒旗舰店,上海,"X >0.3kg, ≤0.5kg",13.287,32,32,22.4,32,32
73,鹅岛啤酒旗舰店,上海,"X >0.5kg, ≤1kg",0.947,1,1,0.7,1,2
74,鹅岛啤酒旗舰店,上海,X ≤0.3kg,2.685,12,12,8.4,12,12


In [45]:
df_pivot_order_handing.to_excel(os.path.join(output,"订单处理费pivot"+input_file))